In [6]:
import os
import uuid
from argparse import ArgumentParser
from pathlib import Path
import shutil
import hydra
from omegaconf import DictConfig

import torchaudio

from utils import VideoIDExtractor, Down_Audio, AudioSeparator, Speaker_verification, convert_audio
from scripts.Audio_Slicer import Slicer
from scripts.whisper import Whisper_ASR
from scripts.Single_speaker_detection import is_single_speaker

In [7]:
@hydra.main(version_base=None, config_path="conf", config_name="config.yaml")
def main(args: DictConfig):
    download_dir='temp/youtube'
    os.makedirs(download_dir, exist_ok=True)
    out='temp/seg_audio1'
    os.makedirs(out, exist_ok=True)
    single_speaker_dir='temp/single_speaker'
    Speaker_verification_dir='temp/Speaker_verificationr'
    os.makedirs(single_speaker_dir, exist_ok=True)
    os.makedirs(Speaker_verification_dir, exist_ok=True)
    sr16K_dir='temp/16K_dir'
    os.makedirs(sr16K_dir, exist_ok=True)
    sr22K_dir='result'
    os.makedirs(sr22K_dir, exist_ok=True)

    ASR_file=Path(sr22K_dir)/'dataset.txt'
    error_log_file=Path(sr22K_dir)/'error_log.txt'


解析视频ID

In [8]:
Pick_id=VideoIDExtractor()
video_ids=Pick_id.process_file_list(args.base.videoUrl)
print(type(video_ids))
print(video_ids)

NameError: name 'args' is not defined

In [ ]:
down_file=[]
for video_id in video_ids:
    audio_path=Down_Audio(video_id=video_id,output_folder=download_dir)
    down_file.append(audio_path)
print(down_file)

In [ ]:
vocals=[]
a=AudioSeparator()
for file in down_file:
    vocal,sr=a.separate_audio(file)
    vocals.append((vocal,sr))

In [ ]:
for vocal in vocals:
    voc,sr=vocal
    slicer = Slicer(
        sr=sr,
        threshold=-30,
        min_length=5000,
        min_interval=300,
        hop_size=20,
        max_sil_kept=5000
    )
    chunks = slicer.slice(voc)
    
    for i, chunk in enumerate(chunks):
        # 构建输出文件路径
        output_file = os.path.join(out, f'{uuid.uuid4()}.wav')
        # 将静音段保存为 WAV 文件
        torchaudio.save(output_file, chunk, sr)

In [ ]:
audio_files=list(Path(out).glob('**/*.wav'))
for i,audio_file_path in enumerate(audio_files):
    result =is_single_speaker(audio_file_path, model_name, use_auth_token, use_cuda)
    if result[0]:
        shutil.copy2(audio_file_path, single_speaker_dir)
        print(f'已经将{audio_file_path}复制到{single_speaker_dir}')

In [ ]:
audio_files=list(Path(single_speaker_dir).glob('**/*.wav'))
for i,audio_file_path in enumerate(audio_files): 

    convert_audio(input_file=audio_file_path,output_file=single_speaker_dir/audio_file_path.name,target_sr=16000,target_channels=1)

In [ ]:
Speaker_verification(single_speaker_dir,Speaker_verification_dir,'temp/f1a6e541-741d-4b64-af3a-575af5a99305.wav')

In [ ]:
#修改名字
files=os.listdir(Speaker_verification_dir)
# [os.rename(os.path.join(Speaker_verification_dir, filename), os.path.join(Speaker_verification_dir, f"p120-{file_counter:04}.wav")) for file_counter, filename in enumerate(files) if filename.endswith(".wav")]
[Path(Speaker_verification_dir).joinpath(f"p125-{file_counter:04}.wav").rename(Path(Speaker_verification_dir).joinpath(filename)) for file_counter, filename in enumerate(files) if filename.endswith(".wav")]


#转换采样率，为whisper做准备
# audio_files=glob.glob(os.path.join(Speaker_verification_dir,'**/*.wav'), recursive=True)
audio_files=list(Path(Speaker_verification_dir).glob('**/*.wav'))
for i,audio_file_path in enumerate(audio_files): 
    convert_audio(input_file=audio_file_path,output_file=sr22K_dir/audio_file_path.name,target_sr=22050,target_channels=1)
    convert_audio(input_file=audio_file_path,output_file=sr16K_dir/audio_file_path.name,target_sr=16000,target_channels=1)